# Summative Assignment 1
## Task 1
This task requires reading books.csv and bookloans.csv into nested lists or dictionaries and ensuring
to have a list of valid book loans. It requires processing to produce a printed report. Considering the validity of the data used in the report or the statistics will be wrong.
A report of books borrowed in 2023 is required listing the returned book number, book title, author,
and the number of days borrowed. This should be a valid list excluding the false books and non-
returned borrows.


We want to identify and show the least popular and most popular books.


### 1. Read CSV Files:

Read books.csv and bookloans.csv into nested lists or dictionaries

In [ ]:
import csv

# we create function that reads a csv file, appends it to a list
def read_csv(file_path):
    """
    Read a CSV file and return its data as a nested list.

    Args:
    - file_path (str): The path to the CSV file.

    Returns:
    - list of lists: The CSV data represented as a nested list.
    """
    data = []
    with open(file_path, 'r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        for row in reader:
            data.append(row)
    return data

# Calling the function to read books.csv
books_data = read_csv('./books.csv')
## Printing the result
print(books_data)

# Calling the function to read bookloans.csv
bookloans_data = read_csv('./bookloans.csv')
print("Bookloans : \n", bookloans_data)


### 2. Process Data from bookloans.csv:

a. Filter out invalid loan transactions. \
b. Calculate the number of days each book was borrowed. \
c. Count the number of times each book was borrowed.



In [ ]:
''' we create a function to filter out invalid loan transaction:
 Invalid transactions are those that were not returned during the year 2023 or have a date_of_return of 0.
  We'll only consider transactions with valid return dates in 2023. '''

def process_loan_data(loan_data):
    """
    Process loan data to filter out invalid transactions, calculate days borrowed,
    and count the number of times each book was borrowed.

    Args:
    - loan_data (list of lists): Loan data as a nested list.

    Returns:
    - dict: A dictionary containing book numbers as keys and their statistics as values.
            The statistics include total number of borrowings and total days borrowed.
    """
    book_stats = {}
    for transaction in loan_data:
        book_number, _, date_of_loan, date_of_return = transaction

        # Convert date strings to integers
        date_of_loan = int(date_of_loan)
        date_of_return = int(date_of_return)

        # Check if transaction is returned during 2023
        if date_of_return != 0 and date_of_return >= 44927 and date_of_return <= 45291:
            # we calculate the number of days each book was borrowed.
            # Calculate days borrowed
            days_borrowed = date_of_return - date_of_loan
            # Update book statistics
            if book_number not in book_stats:
                book_stats[book_number] = {'borrow_count': 1, 'total_days_borrowed': days_borrowed}
            else:
                book_stats[book_number]['borrow_count'] += 1
                book_stats[book_number]['total_days_borrowed'] += days_borrowed
    # Process loan data
    # we check for transaction with a date of return = 0 and filter is out
    for row in loan_data:
        if row[3] != '0':
            return book_stats




book_statistics = process_loan_data(bookloans_data)
print(book_statistics)
#
for book_number, stats in book_statistics.items():
    print(f"Book {book_number}: Borrowed {stats['borrow_count']} times, Total days borrowed: {stats['total_days_borrowed']}")


## 3. Generate Report:
Create a report listing returned book number, title, author, and number of days borrowed.
Sort the report by reverse order of popularity (number of days borrowed).

In [ ]:
def fetch_book_info(book_number, books_data):
    """
    Fetch book information (title and author) based on book number.
    Args:
    - book_number (str): The book number.
    - books_data (list of lists): Data from books.csv as a nested list.

    Returns:
    - tuple: A tuple containing title and author of the book.
    """
    for book in books_data:
        if book[0] == book_number:
            return book[1], book[2]
    return None, None

def generate_report(book_statistics, books_data):
    """
    Generate a report listing returned book number, title, author, and number of days borrowed,
    sorted by reverse order of popularity (number of days borrowed).

    Args:
    - book_statistics (dict): Statistics for each book containing number of borrowings and total days borrowed.
    - books_data (list of lists): Data from books.csv as a nested list.

    Returns:
    - list of tuples: A list of tuples containing book number, title, author, and number of days borrowed.
    """
    report_data = []
    for book_number, stats in book_statistics.items():
        title, author = fetch_book_info(book_number, books_data)
        if title and author:  # If book information is available
            report_data.append((book_number, title, author, stats['total_days_borrowed']))

    # Sort report data by total days borrowed in reverse order
    report_data.sort(key=lambda x: x[3], reverse=True)

    return report_data

# Generate report
report = generate_report(book_statistics, books_data)

# Print report
for book in report:
    print(f"Book Number: {book[0]}, Title: {book[1]}, Author: {book[2]}, Days Borrowed: {book[3]}")

# Task 2
The library is keen to know the interests of its readers to influence purchasing decisions. The books
have different genres. You cannot assume all genres were borrowed; we are only interested in those
that were.

This task involve producing a popularity report of all genres of books borrowed in 2023 and how
many books are in that genre.


To solve Task 2, we will need to:

## 1.  Extract the genres from the books.csv data.

In [ ]:
''' This function extracts unique genres from the books data.
 It iterates over the books data,
  splits the genre strings (assuming they are separated by commas),
and adds them to a set to ensure uniqueness. '''

def extract_genres(books_data):
    """
    Extract genres from the books data.

    Args:
    - books_data (list of lists): Data from books.csv as a nested list.

    Returns:
    - list: A list of unique genres extracted from the books data.
    """
    genres = set()
    for book in books_data[1:]:  # Skip header row
        genres.update(book[3].split(','))  # Assuming genres are separated by commas
    return list(genres)

genres = extract_genres(books_data)
print(genres)

## 2. Count the number of books borrowed in each genre.

In [ ]:
''' This function counts the number of books borrowed in each genre.
 It iterates over the book statistics,
 retrieves the genres for each book from the books data,
and increments the count for each genre accordingly. '''
def count_books_borrowed_by_genre(genres, book_statistics, books_data):
    """
    Count the number of books borrowed in each genre.

    Args:
    - genres (list): A list of unique genres.
    - book_statistics (dict): Statistics for each book containing number of borrowings and total days borrowed.
    - books_data (list of lists): Data from books.csv as a nested list.

    Returns:
    - dict: A dictionary containing genres as keys and the number of books borrowed in each genre as values.
    """
    genre_counts = {genre: 0 for genre in genres}
    for book in books_data[1:]:  # Skip header row
        _, _, _, genre_str, _, _, = book
        book_genres = genre_str.split(',')
        for genre in book_genres:
            genre_counts[genre] += 1
    return genre_counts

genre_counts = count_books_borrowed_by_genre(genres, book_statistics, books_data)
print(genre_counts)




## 3. Generate Report:

We create a report listing each genre and the number of books borrowed in that genre and sort the report as needed.

In [ ]:
''' This function generates a report listing each genre along with the number of books borrowed in that genre.
 It sorts the report data based on the count of books borrowed in each genre, in reverse order.'''

def generate_genre_report(genre_counts):
    """
    Generate a report listing each genre and the number of books borrowed in that genre.

    Args:
    - genre_counts (dict): A dictionary containing genres as keys and the number of books borrowed in each genre as values.

    Returns:
    - list of tuples: A list of tuples containing genre and the number of books borrowed in that genre.
    """
    report_data = [(genre, count) for genre, count in genre_counts.items()]
    # Sort report data as needed (e.g., by count)
    report_data.sort(key=lambda x: x[1], reverse=True)
    return report_data
# Generate genre report
genre_report = generate_genre_report(genre_counts)
## finally, we print the genre report containing the required information for each genre.
for genre, count in genre_report:
  print(f"Genre: {genre}, Number of Books Borrowed: {count}")

In [ ]:
print(bookloans_data)

In [ ]:
def calculate_loan_statistics(book_statistics):
    """
    Calculate loan statistics from book statistics.

    Args:
    - book_statistics (dict): Dictionary containing statistics for each book.

    Returns:
    - dict: Dictionary containing loan statistics.
    """
    num_loans = sum(stats['borrow_count'] for stats in book_statistics.values())
    total_days_borrowed = sum(stats['total_days_borrowed'] for stats in book_statistics.values())

    num_late_returns = sum(1 for stats in book_statistics.values() if stats['total_days_borrowed'] > 14)
    total_days_late = sum(max(0, stats['total_days_borrowed'] - 14) for stats in book_statistics.values() if stats['total_days_borrowed'] > 14)

    average_loan_duration = total_days_borrowed / num_loans if num_loans > 0 else 0
    proportion_late_returns = num_late_returns / num_loans if num_loans > 0 else 0
    average_late_period = total_days_late / num_late_returns if num_late_returns > 0 else 0

    loan_statistics = {
        'Number of loans': num_loans,
        'Total days borrowed': total_days_borrowed,
        'Number of late returns': num_late_returns,
        'Total days late': total_days_late,
        'Average loan duration': average_loan_duration,
        'Proportion of late returns': proportion_late_returns,
        'Average late period': average_late_period
    }

    return loan_statistics

# Example usage:
loan_statistics = calculate_loan_statistics(book_statistics)
for key, value in loan_statistics.items():
    print(f"{key}: {value}")
